#### colab requirements

In [1]:
## ONLY on colab
# !pip install transformers
# !pip install datasets
# !pip install trl

## ⬇️ Imports

In [33]:
import numpy as np
from tqdm import tqdm

from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer,
    RobertaForSequenceClassification,   # reward model
    RobertaTokenizer,                   # reward model tokenizer
)
from datasets import load_dataset

import torch
from torch.optim import Adam
from trl import (
    AutoModelForCausalLMWithValueHead, 
    PPOConfig, 
    PPOTrainer,
    create_reference_model,
    set_seed,
)
from trl.core import LengthSampler

## 🤖 Model

### Configs

In [34]:
# model_name = 'ybelkada/gpt-j-6b-sharded-bf16'       # has 28 layers
model_name = 'MBZUAI/LaMini-GPT-124M'              # smallest one, having 12 layers
log_with = None                     # or 'wandb'
learning_rate = (1.47e-5) * 2       # default one (?)
mini_batch_size = 4                 # PPO minibatch size
batch_size = 12                     # batchsize (number of element for each reward computed (ppo_trainer.dataloader batchsize))
ppo_epochs = 100
gradient_accumulation_steps = 1     # [TODO]: check what is this!
model_save_path = './checkpoints/'


In [35]:
config = PPOConfig(
    model_name = model_name,
    learning_rate = learning_rate,
    log_with = log_with,
    ppo_epochs = ppo_epochs,
    mini_batch_size = mini_batch_size,
    batch_size = batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    accelerator_kwargs = {'cpu': True}
)

### Dataset
Method that loads the dataset, customize it to train on different data!

In [4]:
def build_dataset(
    config, 
    dataset_name="allenai/real-toxicity-prompts", 
    input_min_text_length = 10, 
    input_max_text_length = 15,
):
    """
    Build dataset for training. This builds the dataset from `load_dataset` and returns 
    a Dataloader class (from torch.utils.data)

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded. (from load_dataset)
    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    # from the PPO trainer
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token

    ds = load_dataset(dataset_name, split = "train")

    # function to filter only prompts that lead to a toxicity score > 0.3
    # in order to increase the chances for the model to generate toxicity prompts
    def filter_fn(sample):
        toxicity = sample["prompt"]["toxicity"]
        return toxicity is not None and toxicity > 0.3

    ds = ds.filter(filter_fn, batched=False)
    
    # only prompts in between input_min_text_length : input_max_text_length
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        prompt = sample["prompt"]["text"]
        continuation = sample["continuation"]["text"]

        sample["input_ids"] = tokenizer.encode(prompt + continuation)[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    # tokenize
    ds = ds.map(tokenize, batched = False)
    # set as torch.utils.data.DataLoader
    ds.set_format(type="torch")

    ds = ds.train_test_split(test_size = 0.2, shuffle = False)["train"]

    return ds

# Getting dataset
min_input_length = 30
max_input_length = 40
dataset = build_dataset(
    config, 
    input_min_text_length = min_input_length, 
    input_max_text_length = max_input_length
)

Found cached dataset json (/Users/daniel/.cache/huggingface/datasets/allenai___json/allenai--real-toxicity-prompts-eb8779dd2693db47/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


Filter:   0%|          | 0/99442 [00:00<?, ? examples/s]

Map:   0%|          | 0/35109 [00:00<?, ? examples/s]

In [40]:
from datasets import Dataset
from rewardlm.data.data_utils import get_DIALOCONAN_prepro

def build_dataset_2(config):
    # from the PPO trainer
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    ds = Dataset.from_dict({
        'text': get_DIALOCONAN_prepro(delete_last_assistant_response = True)
    })

    def tokenize(sample):
        prompt = sample["text"]
        # continuation = sample["continuation"]["text"]

        sample["input_ids"] = tokenizer.encode(prompt)
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample
    ds = ds.map(tokenize, batched = False)
    ds.set_format(type="torch")
    ds = ds.train_test_split(test_size = 0.2, shuffle = False)["train"]
    return ds

my_ds = build_dataset_2(config)


Map:   0%|          | 0/8312 [00:00<?, ? examples/s]

In [5]:
# other stuff
# set seed before initializing value head for deterministic eval
set_seed(config.seed)

# dunno
def collator(data):
    return dict(
        (key, [d[key] for d in data]) for key in data[0]
    )


### Reference model
We load the model in bfloat16 to save memory

In [6]:
# model loaded in float16 to reduce memory usage
model_stock = AutoModelForCausalLM.from_pretrained(
    config.model_name, 
    torch_dtype = torch.bfloat16
)
model = AutoModelForCausalLMWithValueHead.from_pretrained(model_stock)

# GPT-2 / GPT-J tokenizer has a pad token, but it is not eos_token by default. We need to set it to eos_token.
# only for this model.
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
## so ONLY FOR GPT (?)
tokenizer.pad_token = tokenizer.eos_token

# reference model sharing 5 layers out of 12 total layers (for 'gpt-neo-125m')
# reference models are frozen copies of the model that is trained (in eval mode) [from doc]
ref_model = create_reference_model(
    model, 
    num_shared_layers = 5,
    pattern = 'transformer.h.{layer}'
)

# We make sure to use `Adam` optimizer on the model parameters that require gradients.
optimizer = Adam(
    filter(lambda p: p.requires_grad, model.parameters()), 
    lr = config.learning_rate
)

![shared layers](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/trl-shared-layers.png)

### PPO trainer

In [7]:
ppo_trainer = PPOTrainer(
    config,
    model,                          # model to be optimized w/ value head
    ref_model = ref_model,          # reference model used for KL penalty
    tokenizer = tokenizer,          # used for encoding
    dataset = dataset,
    data_collator = collator,
    optimizer = optimizer,
    # lr_scheduler = ,              # <optional> lr scheduler
)

In [13]:
dataset

Dataset({
    features: ['filename', 'begin', 'end', 'challenging', 'prompt', 'continuation', 'input_ids', 'query'],
    num_rows: 28087
})

### Reward model

In [11]:
toxicity_model_id = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = RobertaTokenizer.from_pretrained(toxicity_model_id)
# Also load the toxicity model in fp16 to save memory
toxicity_model = RobertaForSequenceClassification.from_pretrained(
    toxicity_model_id, 
    torch_dtype=torch.float32
).to(ppo_trainer.accelerator.device)

In [9]:
# We then define the arguments to pass to the `generate` function. These arguments
# are passed to the `generate` function of the PPOTrainer, which is a wrapper around
# the `generate` function of the trained model.
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}
output_min_length = 20
output_max_length = 30
output_length_sampler = LengthSampler(output_min_length, output_max_length)

### Train

In [12]:
# import os 
# os.environ[''] = '1'

In [17]:
a = next(iter(ppo_trainer.dataloader))

In [30]:
a['input_ids']

[tensor([   34,  2117, 39485,   338, 13455,    12, 20333,    11, 35101,  3259,
         47334,  4409,   481,  1011,   534,  2748, 13871,   284,   787,  1654,
           534, 33225,   290,    14,   273, 46649,  6107,  6063,   318,  1296,
            12, 32232]),
 tensor([ 2990,   389,  6639,   286,  2042, 14287,   852,  1458,  8375,   416,
          3344,    12,  1493,  1644,    13]),
 tensor([  464, 11375,   286,   465, 38007,   867,   812,  1568,  2921, 15796,
           691,   257,   953, 39901,   286,  1540,   558,    11,   339,  6797,
            11,   329,   339,   991,  5300,   883,  1440,  2250,   351,   683,
           783,    11,  4647,  1568,    11]),
 tensor([  447,   250,  1544,  2058,   572, 13779,   475,   706,   262,   564,
           250, 13602,  1136,  2046,   447,   251,   636,    11,   340,   477,
          2925,   284,  7510,    13,   447,   251]),
 tensor([38202,   648,   126,   240,    82,  1097, 23354,   656,   257, 13990,
           379,   262,  1735,   286,   2

In [24]:
[ele.shape for ele in a['input_ids']]

[torch.Size([32]),
 torch.Size([15]),
 torch.Size([35]),
 torch.Size([26]),
 torch.Size([30]),
 torch.Size([30]),
 torch.Size([38]),
 torch.Size([39]),
 torch.Size([36]),
 torch.Size([37]),
 torch.Size([22]),
 torch.Size([36])]

In [12]:
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    # Get response from the policy model
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
    print('Response obtained')

    # Compute sentiment score # noqa
    texts = batch["response"]
    print('Asking toxicity')
    toxicity_inputs = toxicity_tokenizer(
        texts, 
        padding=True, 
        truncation=True, 
        return_tensors="pt"
    ).to(ppo_trainer.accelerator.device)
    logits = toxicity_model(**toxicity_inputs).logits.float()
    toxicity_labels = (logits[:, 0]).tolist()
    print('Got toxicity')

    rewards = [torch.tensor(output) for output in toxicity_labels]
    print('Reward computed, running PPO')

    # Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)
    print('PPO updated')

    # Save model every 100 epochs
    if epoch % 100 == 0:
        if ppo_trainer.accelerator.is_main_process:
            ppo_trainer.save_pretrained(model_save_path)

0it [00:00, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Response obtained
Asking toxicity
Got toxicity
Reward computed, running PPO


0it [02:14, ?it/s]


KeyboardInterrupt: 